# **SW03: Preprocessing**

In the following, we demonstrate some preprocessing methods for the Titanic 
dataset. The dataset contains the following data about the passengers:

- Passenger ID - Unique identifier of the passenger
- Survived - Did they survive or not? (Target variable)
- P Class - The class of the ticket (1st, 2nd, 3rd)
- Name - Name of the passenger
- Sex - Sex of the passenger (male, female)
- Age - Age of passenger (in years)
- SibSp - Number of siblings or spouse(s) aboard
- Parch - Parent or child aboard
- Ticket - Ticket identifier
- Fare - Amount paid for the ticket
- Cabin - Cabin of residence (if available)
- Embarked - Point of embarkment (C = Cherbourg, Q = Queenstown, S = Southampton)


Based on this data, the goal is to develop a model that predicts whether a
passenger survived or not.

You will see that a logistic regression classifier is able to predict the 
survival of a passanger with a success rate of 0.781. 

However, there's a lot to improve in the data... Do you manage to achieve a better score?

---

In [30]:
# Basic imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# Some Jupyter magic for nicer output
%config InlineBackend.figure_formats = ["svg"]   # Enable vectorized graphics

# Jupyter / IPython configuration:
# Automatically reload modules when modified
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
import ml

from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
# Load titanic dataset
df = pd.read_csv('../data/datasets/titanic.csv', sep=',')
display(df)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## **Drop uninformative columns**

Some columns are not important for the classification, as they are unique 
for each passenger and therefore not informative for the classification: 
PassengerId, Name, Ticket.

Let's drop this features from the dataframe.

In [32]:
columns_to_drop = ['PassengerId', 'Name', 'Ticket']
df = df.drop(columns_to_drop, axis=1)
display(df)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,NaN,S
887,1,1,female,19.0,0,0,30.0000,B42,S
888,0,3,female,NaN,1,2,23.4500,NaN,S
889,1,1,male,26.0,0,0,30.0000,C148,C


## **Handling of Missing Values**

In [33]:
# We can use df.info() to see which columns contain NaN values. We see that
# the columns 'Age', 'Cabin' and 'Embarked' contain missing values. 
print("######################################")
df.info()
print("######################################\n")

# Check which rows contain NaN values (NaN = Not a Number)
row_has_nan = pd.isnull(df).any(axis=1)
print(f'Number of rows that contain at least one NaN value: {sum(row_has_nan)}')

# We can use row_has_nan to drop these rows from the dataframe 
# Remember: ~mask inverts a boolean mask: True -> False, False -> True
df_clean = df[~row_has_nan]

print(f'Number of rows before cleaning: {len(df)}')
print(f'Number of rows after cleaning: {len(df_clean)}')

######################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB
######################################

Number of rows that contain at least one NaN value: 708
Number of rows before cleaning: 891
Number of rows after cleaning: 183


In [34]:
######################
###   EXERCISE 1   ###
######################

# After cleaning the dataset, we see that there are only 183 remaining rows.
# Most of our data got lost in the process! Can you improve the result of the
# classifier with imputation?

from sklearn.impute import SimpleImputer
df_clean = df.copy()

#hier werden alle NAN Werte mit dem median der Kolonne bestummen und aufgefüllt
imputer = SimpleImputer(strategy = 'median')
columns_to_impute = ['Age']
imputer.fit(df[columns_to_impute])
df_clean[columns_to_impute] = imputer.transform(df[columns_to_impute])


#hier werden alle NAN Werte mit den most frequent Daten aufgefüllt
imputer = SimpleImputer(strategy = 'most_frequent')
columns_to_impute = ['Embarked']
imputer.fit(df[columns_to_impute])
df_clean[columns_to_impute] = imputer.transform(df[columns_to_impute])

#selbe wie oben
imputer = SimpleImputer(strategy = 'most_frequent')
columns_to_impute = ['Cabin']
imputer.fit(df[columns_to_impute])
df_clean[columns_to_impute] = imputer.transform(df[columns_to_impute])



## **Separate target variable**

In [35]:
# The column "Survived" contains our target values, we should thus 
# remove it from the dataframe and store it in a separate variable y.
y = df_clean['Survived']
df_clean = df_clean.drop(['Survived'], axis=1)

# Exercise: What is the difference between df['Survived'] and df[['Survived']]?
# Solution: df['Survived'] returns a pandas Series, while 
#           df[['Survived']] returns a DataFrame with one column.

## **Encoding of Categorical Data**

In [36]:
df_clean['Cabin'] = df_clean['Cabin'].str.replace(r'\d+', '', regex=True)

In [37]:
df_clean['Cabin'].unique()

array(['B B', 'C', 'E', 'G', 'D', 'A', 'C C C', 'B', 'F', 'F G', 'D D',
       'F E', 'C C', 'B B B B', 'T', 'B B B'], dtype=object)

In [38]:
# Now we want to one-hot-encode the columns that are not yet numeric, 
# because our machine learning classifiers can only handle numeric data.
# categorical_columns = ['Sex', 'Cabin', 'Embarked']
categorical_columns = ['Sex', 'Embarked', 'Cabin']

# For this we split the dataframe into data that is already numeric and 
# data that has to be encoded: 
df_categorical = df_clean[categorical_columns]
df_numeric = df_clean.drop(categorical_columns, axis=1)


In [39]:
######################
###   EXERCISE 2   ###
######################

# Use the OneHotEncoder from sklearn to encode the categorical columns.

from sklearn.preprocessing import OneHotEncoder

# Erstellen eines OneHotEncoders
encoder = OneHotEncoder(sparse_output=False)

# Anwenden des Encoders
df_categorical_encoded = encoder.fit_transform(df_categorical)

# Die kodierten Spalten in einen neuen DataFrame umwandeln
df_categorical_encoded = pd.DataFrame(df_categorical_encoded, columns=encoder.get_feature_names_out(categorical_columns))





## **Normalize the numeric data**

In [41]:
df_numeric

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500
...,...,...,...,...,...
886,2,27.0,0,0,13.0000
887,1,19.0,0,0,30.0000
888,3,28.0,1,2,23.4500
889,1,26.0,0,0,30.0000


In [45]:
######################
###   EXERCISE 3   ###
######################

# The numeric data in the dataframe df_numeric is not yet normalized. This can
# lead to problems when training machine learning models and affect the 
# predictive performance of the model. Apply one of the scaling methods 
# we discussed to normalize (or standardize) the data. Does it help to improve
# the performance of the model?


from sklearn.preprocessing import StandardScaler

# Erstellen des Scalers
scaler = StandardScaler()

# Anwenden des Scalers auf die numerischen Daten
df_numeric_scaled = scaler.fit_transform(df_numeric)

# Optional: Den skalierten DataFrame in einen Pandas DataFrame umwandeln (falls benötigt)
df_numeric_scaled = pd.DataFrame(df_numeric_scaled, columns=df_numeric.columns)

print(df_numeric_scaled)







       Pclass       Age     SibSp     Parch      Fare
0    0.827377 -0.565736  0.432793 -0.473674 -0.502445
1   -1.566107  0.663861  0.432793 -0.473674  0.786845
2    0.827377 -0.258337 -0.474545 -0.473674 -0.488854
3   -1.566107  0.433312  0.432793 -0.473674  0.420730
4    0.827377  0.433312 -0.474545 -0.473674 -0.486337
..        ...       ...       ...       ...       ...
886 -0.369365 -0.181487 -0.474545 -0.473674 -0.386671
887 -1.566107 -0.796286 -0.474545 -0.473674 -0.044381
888  0.827377 -0.104637  0.432793  2.008933 -0.176263
889 -1.566107 -0.258337 -0.474545 -0.473674 -0.044381
890  0.827377  0.202762 -0.474545 -0.473674 -0.492378

[891 rows x 5 columns]


## **Merge categorical and numeric data**

In [46]:
# We have now preprocessed the categorical and numeric data.
# Let's merge this data back together.
X = pd.concat([df_numeric_scaled, df_categorical_encoded], axis=1)
X

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,...,Cabin_C C,Cabin_C C C,Cabin_D,Cabin_D D,Cabin_E,Cabin_F,Cabin_F E,Cabin_F G,Cabin_G,Cabin_T
0,0.827377,-0.565736,0.432793,-0.473674,-0.502445,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.566107,0.663861,0.432793,-0.473674,0.786845,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.827377,-0.258337,-0.474545,-0.473674,-0.488854,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.566107,0.433312,0.432793,-0.473674,0.420730,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.827377,0.433312,-0.474545,-0.473674,-0.486337,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,-0.369365,-0.181487,-0.474545,-0.473674,-0.386671,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
887,-1.566107,-0.796286,-0.474545,-0.473674,-0.044381,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
888,0.827377,-0.104637,0.432793,2.008933,-0.176263,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
889,-1.566107,-0.258337,-0.474545,-0.473674,-0.044381,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **Model training and evaluation**

Using the preprocessed data, let's train a logistic regression model.

To properly evaluate the performance of our model, we split the data into a
training and a test set. We want to use 30% of the data for testing and the 
rest for training.

In [16]:
######################
###   EXERCISE 4   ###
######################

# Train a logistic regression model on the data and evaluate its performance.
# Apply the following steps:
#   - Split the data into a training and a test set (70% training, 30% test)
#   - Train the logistic regression model on the training set
#   - Predict the target values on the test set
#   - Evaluate the performance of the model using the accuracy_score()
#     function from sklearn.metrics.
#
# Report the accuarcy of the model.